# **Modelo de Classificação para Detecção de Fraudes em Cartões de Crédito**

Data: 18/09 - Autor: Eduardo Oliveira



In [38]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [39]:
# Definindo o número de amostras.
total_amostras = 10000
fraud_ratio = 0.05 # 5% de fraudes.
fraud_samples = int(total_amostras * fraud_ratio)
normal_samples = total_amostras - fraud_samples



In [40]:
# Gerar dados sintéticos com o make_classification
X, y = make_classification(n_samples=total_amostras, n_features=5, weights=[0.95, 0.05], n_informative=2, n_redundant=0, random_state=42)

In [41]:
df = pd.DataFrame(X, columns=['valor_transacao', 'latitude', 'longitude', 'historico_usuario', 'horario_compra'])
df['fraude'] = y # Rótulo de fraude

In [42]:
from sklearn.preprocessing import MinMaxScaler

def ajustar_valores(df):
    # Cria uma cópia para não alterar o dataframe original durante as operações
    df_ajustado = df.copy()
    
    # Transforma os valores para serem sempre positivos e os multiplica por 1000
    df_ajustado['valor_transacao'] = np.abs(df_ajustado['valor_transacao']) * 1000

    # Usa MinMaxScaler para mapear os valores originais para as faixas corretas
    scaler = MinMaxScaler(feature_range=(-90, 90))
    df_ajustado['latitude'] = scaler.fit_transform(df_ajustado[['latitude']])
    
    scaler = MinMaxScaler(feature_range=(-180, 180))
    df_ajustado['longitude'] = scaler.fit_transform(df_ajustado[['longitude']])

    # Mapeia para faixas de inteiros realistas
    scaler = MinMaxScaler(feature_range=(1, 50)) # Ex: histórico de 1 a 50 transações
    df_ajustado['historico_usuario'] = scaler.fit_transform(df_ajustado[['historico_usuario']]).astype(int)

    scaler = MinMaxScaler(feature_range=(0, 23)) # Ex: hora de 0 a 23
    df_ajustado['horario_compra'] = scaler.fit_transform(df_ajustado[['horario_compra']]).astype(int)
    
    return df_ajustado

In [43]:
df = ajustar_valores(df)

In [44]:
# Salvar como CSV
df.to_csv('datasets/dados_transacoes.csv', index=False)
print("Dados sintéticos salvos em 'datasets/dados_transacoes.csv'") 

Dados sintéticos salvos em 'datasets/dados_transacoes.csv'


## Análise Exploratória de Dados (EDA)

In [45]:
df.head()

,valor_transacao,latitude,longitude,historico_usuario,horario_compra,fraude
0,251.421786,-5.666542,83.083258,27,16,0
1,1966.463062,-32.024210,9.577996,25,6,0
2,249.303784,-13.168819,81.220209,31,6,0
3,183.368003,4.523139,-29.620250,28,14,0
4,507.164884,0.533369,96.639386,27,6,0


In [46]:
df.isnull().sum()

valor_transacao      0
latitude             0
longitude            0
historico_usuario    0
horario_compra       0
fraude               0
dtype: int64

In [47]:
df.shape

(10000, 6)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   valor_transacao    10000 non-null  float64
 1   latitude           10000 non-null  float64
 2   longitude          10000 non-null  float64
 3   historico_usuario  10000 non-null  int64  
 4   horario_compra     10000 non-null  int64  
 5   fraude             10000 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 468.9 KB


## **Preparação dos Dados**

**Normalização dos dados**

In [49]:

scaler = StandardScaler()
df[['valor_transacao', 'historico_usuario', 'horario_compra']] = scaler.fit_transform(df[['valor_transacao', 'historico_usuario', 'horario_compra']]) 
df.head()


,valor_transacao,latitude,longitude,historico_usuario,horario_compra,fraude
0,-0.906949,-5.666542,83.083258,-0.082325,1.686569,0
1,1.929172,-32.024210,9.577996,-0.410182,-1.437980,0
2,-0.910451,-13.168819,81.220209,0.573390,-1.437980,0
3,-1.019488,4.523139,-29.620250,0.081604,1.061659,0
4,-0.484033,0.533369,96.639386,-0.082325,-1.437980,0


In [50]:
# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('fraude', axis=1), df['fraude'], test_size=0.3, random_state=42, stratify=df['fraude'])  

In [51]:
# Escolher modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
# Treinar modelo
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [52]:
# Avaliar modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2838
           1       0.81      0.51      0.63       162

    accuracy                           0.97      3000
   macro avg       0.89      0.75      0.81      3000
weighted avg       0.96      0.97      0.96      3000



In [53]:
import pickle
# Salvar o modelo treinado
filename = 'modelo/modelo_fraude.pkl'
pickle.dump(model, open(filename, 'wb'))

print(f"Modelo salvo em '{filename}'")

Modelo salvo em 'modelo/modelo_fraude.pkl'
